# A First Neural Network on Graph
=========================

This lab is based on https://docs.dgl.ai/tutorials/basics/1_first.html

DGL is a Python package dedicated to deep learning on graphs, built atop
existing tensor DL frameworks (e.g. Pytorch, MXNet) and simplifying the
implementation of graph-based neural networks.

The goal of this tutorial:

- Understand how DGL enables computation on graph from a high level.
- Train a simple graph neural network in DGL to classify nodes in a graph.

It is composed of 6 steps:
- Step 1: Creating a graph in DGL
- Step 2: Assign features to nodes or edges
- Step 3: Define a Graph Convolutional Network (GCN)
- Step 4: Data preparation and initialization
- Step 5: Train then visualize
- Step 6: Create your own GCN layer

In [ ]:
# You need to install the DGL library
# With Anaconda, just use the Anaconda Navigator or conda install -c dglteam dgl
# With Google Colab, run the following command line
!pip install dgl

In [ ]:
%matplotlib inline

## Tutorial problem description
--------------------------------------------------

The tutorial is based on the "Zachary's karate club" problem. The karate club
is a social network that includes 34 members and documents pairwise links
between members who interact outside the club.  The club later divides into
two communities led by the instructor (node 0) and the club president (node
33). The network is visualized as follows with the color indicating the
community:

![](https://data.dgl.ai/tutorial/img/karate-club.png)


The task is to predict which side (0 or 33) each member tends to join given
the social network itself.

A more detailled description is available on https://en.wikipedia.org/wiki/Zachary%27s_karate_club



## Step 1: Creating a graph in DGL
--------------------------------------------
Create the graph for Zachary's karate club as follows:



### Question 1: what is "dgl.graph()"?

#### Answer:
DGL represents each node by a unique integer, called its node ID, and each edge by a pair of integers corresponding to the IDs of its end nodes. DGL assigns to each edge a unique integer, called its edge ID, based on the order in which it was added to the graph. The numbering of node and edge IDs starts from 0. In DGL, all the edges are directed, and an edge (𝑢,𝑣) indicates that the direction goes from node 𝑢 to node 𝑣.

There are different ways to add nodes and edges. Here, we define the edges and the list of edges is the input of dgl.graph.

### Question 2: how DGL is modelling undirected graph?

#### Answer:

The function considers all the couples (src, dst) and (dst, src). It follows that two arrows (directed edge) are assigned to each couple of nodes. Hence, an undirected edge is modeled as a couple of directed edges.

In [ ]:
import dgl
import numpy as np

print(dgl.__version__)

Nnodes = 34

def build_karate_club_graph():
    # All 78 edges are stored in two numpy arrays. One for source endpoints
    # while the other for destination endpoints.
    src = np.array([1, 2, 2, 3, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, 8, 9, 10, 10,
        10, 11, 12, 12, 13, 13, 13, 13, 16, 16, 17, 17, 19, 19, 21, 21,
        25, 25, 27, 27, 27, 28, 29, 29, 30, 30, 31, 31, 31, 31, 32, 32,
        32, 32, 32, 32, 32, 32, 32, 32, 32, 33, 33, 33, 33, 33, 33, 33,
        33, 33, 33, 33, 33, 33, 33, 33, 33, 33])
    dst = np.array([0, 0, 1, 0, 1, 2, 0, 0, 0, 4, 5, 0, 1, 2, 3, 0, 2, 2, 0, 4,
        5, 0, 0, 3, 0, 1, 2, 3, 5, 6, 0, 1, 0, 1, 0, 1, 23, 24, 2, 23,
        24, 2, 23, 26, 1, 8, 0, 24, 25, 28, 2, 8, 14, 15, 18, 20, 22, 23,
        29, 30, 31, 8, 9, 13, 14, 15, 18, 19, 20, 22, 23, 26, 27, 28, 29, 30,
        31, 32])
    # Edges are directional in DGL; Make them bi-directional.
    u = np.concatenate([src, dst])
    v = np.concatenate([dst, src])
    # Construct a DGL graph
    return dgl.graph((u, v))

Print out the number of nodes and edges in our newly constructed graph:



In [ ]:
G = build_karate_club_graph()
print('We have %d nodes.' % G.number_of_nodes())
print('We have %d edges.' % G.number_of_edges())

Visualize the graph by converting it to a networkx graph.
The library networkx is described on https://networkx.github.io/documentation/stable/



In [ ]:
import networkx as nx
# Since the actual graph is undirected, we convert it for visualization purpose.
nx_G = G.to_networkx().to_undirected()
# Kamada-Kawaii layout usually looks pretty for arbitrary graphs
pos = nx.kamada_kawai_layout(nx_G)
nx.draw(nx_G, pos, with_labels=True, node_color=[[.7, .7, .7]])

### Question 3: print the adjency matrix of the graph?

In [ ]:
#Fill in this cell
print(G.adjacency_matrix()) # this is a sparse matrix: the indices correspond to matrix indices (location of non-zero coefficients)
A = G.adjacency_matrix().to_dense() # A is the adjency matrix as a dense tensor. It is a 34x34 tensor of 0 or 1 values.
print(A.shape)
print(A)

### Question 4: create a DGL graph g with 6 nodes and the directed edges 0-1, 1->3, 2->4, 3->5

Warning: The graph g is only used in this question. The rest of the lab concerns the graph G.

In [ ]:
# Fill in the cell
g = dgl.graph([])
g.add_nodes(6)  # 6 isolated nodes are added
g.add_edges(0, 1)
g.add_edges([1, 2, 3], [3, 4, 5])  # three edges: 1->3, 2->4, 3->5
print(g.adjacency_matrix())
A = g.adjacency_matrix().to_dense() # A is the adjency matrix as a dense tensor. It is a 34x34 tensor of 0 or 1 values.
print(A)


## Step 2: Assign features to nodes or edges
--------------------------------------------
Graph neural networks associate features with nodes and edges for training.
For our classification example, we assign each node an input feature as a one-hot vector:
node $v_i$'s feature vector is $[0,\ldots,1,\dots,0]$,
where the $i^{th}$ position is one.

In DGL, you can add features for all nodes at once, using a feature tensor that
batches node features along the first dimension. The code below adds the one-hot
feature for all nodes:



### Question 5: is the name "feature" crucial in the following cell? Can we replace it by an other name?

#### Answer:

The name does not matter. Yes, we can use an other name. Of coure, when a name has been used, we must use it again to retrieve its value. 

"ndata" for node data.

You can also add data to the edges with "G.edata[...]".

In [ ]:
import torch
import torch.nn as nn

G.ndata['feature'] = torch.eye(Nnodes)

### Question 6: create a new feature named "featureRandom" which associates a random normal vector of size 6 to each node. The vector must be generated with Pytorch.

In [ ]:
# Fill in the cell
G.ndata['featureRandom'] = torch.randn(34, 6)

Print out the node features to verify:



In [ ]:
# print out node 2's input feature
print(G.nodes[2].data['featureRandom'])

# print out node 10 and 11's input features
print(G.nodes[[10, 11]].data['featureRandom'])

# After assignments, each node or edge field will be associated with a scheme containing the shape and data type 
# (dtype) of its field value.
print(G.node_attr_schemes())

### Question 7: Compute a node feature 'deg' that contains the out degree (number of outgoing edges) of the node

In [ ]:
# Fill in the cell
print(G.nodes())
G.ndata['deg'] = G.out_degrees(G.nodes()).float()
print(G.nodes[:].data['deg'])

### Question 8: remove all the node features. 

In [ ]:
# Fill in the cell

# You can also remove node or edge states from the graph. 
# This is particularly useful to save memory during inference.
G.ndata.pop('feature')
G.ndata.pop('featureRandom')
G.ndata.pop('deg')
print(G.node_attr_schemes())



## Step 3: Define a Graph Convolutional Network (GCN)
--------------------------------------------------
To perform node classification, use the Graph Convolutional Network
(GCN) developed by [Kipf and Welling](https://arxiv.org/abs/1609.02907) 

Here is the simplest definition of a GCN framework. We recommend that you 
read the original paper for more details.





### Question 9: give a mathematical description of the folllowing model GCN. What is the output of the neural network?

#### Answer:

- At layer $l$, each node $v_i^l$ carries a feature vector $h_i^l$.
- Each layer of the GCN tries to aggregate the features from $u_i^{l}$ where
  $u_i$'s are neighborhood nodes to $v$ into the next layer representation at
  $v_i^{l+1}$. This is followed by an affine transformation with some
  non-linearity.

The above definition of GCN fits into a **message-passing** paradigm: Each
node will update its own feature with information sent from neighboring
nodes. A graphical demonstration is displayed below.

![mailbox](https://data.dgl.ai/tutorial/1_first/mailbox.png)

The GraphConv module implements one Graph Convolutional layer.

In [ ]:
from dgl.nn.pytorch import GraphConv

import torch.nn.functional as F

# Define a 2-layer GCN model
class GCN(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, hidden_size)
        self.conv2 = GraphConv(hidden_size, num_classes)

    def forward(self, g, inputs):
        h = self.conv1(g, inputs)
        h = torch.relu(h)
        h = self.conv2(g, h)
        return h



## Step 4: Data preparation and initialization
-------------------------------------------

We use one-hot vectors to initialize the node features. Since this is a
semi-supervised setting, only the instructor (node 0) and the club president
(node 33) are assigned labels. The implementation is available as follow.



### Question 10: what is the role of the Embedding in the following?

#### Answer:

It is a classical input embedding that linearly transforms some categorical inputs into numerical vectors. The categorical features are first encoded as one-hot vectors and then each one-hot vector is linearly transformed.

In [ ]:
embed_dim = 3
embed = nn.Embedding(Nnodes, embed_dim)  # 34 nodes with embedding dim equal to 3
inputs = embed.weight

In [ ]:
labeled_nodes = torch.tensor([0, 33])  # only the instructor and the president nodes are labeled
labels = torch.tensor([0, 1])  # their labels are different

In [ ]:
# The first layer transforms input features of size of 'embed_dim' to a hidden size of 'hidden_dim'.
# The second layer transforms the hidden layer and produces output features of
# size 2, corresponding to the two groups of the karate club.
hidden_dim = 10
net = GCN(embed_dim, hidden_dim, 2)

## Step 5: Train then visualize
----------------------------
The training loop is exactly the same as other PyTorch models.
We (1) create an optimizer, (2) feed the inputs to the model,
(3) calculate the loss and (4) use autograd to optimize the model.



### Question 11: what is the optimization algorithm used for training the neural network? Why are we using "itertools"?

#### Answer:

We use the Adam algorithm.

It must update both the neural network parameters and the embedding layer parameters. 

"itertools.chain" chains (concatenates) two iterable objects. chain yields the elements of the first iterator until it gets exhausted, and then it yields the elements of the second one. In your code chain puts together the parameters of the two generators so they will be optimized simultaneously. Used for treating consecutive sequences of parameters as a single sequence of parameters.

In [ ]:
import itertools

optimizer = torch.optim.Adam(itertools.chain(net.parameters(), embed.parameters()), lr=0.01)

### Question 12: what is the role of "log_softmax" in the following cell? What is the role of "nll_loss"?

#### Answer:

"log_softmax" is computing the logarithm of the softmax of the neural network layer.

"nll_loss" is the negative log likelihood loss.

Let $f$ be a vector containing the class scores for a single example, that is, the output of the network. Thus $f_k$ is an element for a certain class $k$.

We can then rewrite the softmax output as
$$
p_k=\frac{e^{f_k}}{\sum{j=1}^{K}e^{f_j}}
$$
where $K$ is the number of classes.

The negative log-likelihood is
$$
L_i = -\log(p_{y_i})
$$
where $y_i$ is the correct class index of input sample $i$.

The loss $L_i$ is positive. It is minimum (equal to $0$) when $p_{y_i}=1$ (good classification).

In [ ]:
# Just a cell to illustrate how nll_loss is working

# input is of size N x C = 3 x 5
# we have 5 classes
input = torch.randn(3, 5, requires_grad=True)
# each element in target has to have 0 <= value < C
target = torch.tensor([1, 0, 4])
output = F.nll_loss(F.log_softmax(input,1), target) # WARNING: logsoftmax is crucial!
# The log in logsoftmax is clearly essential ; if we remove the log, it does not work anymore
# You can check that by uncommenting the following line and the line Q=-P
#output = F.nll_loss(F.softmax(input,1), target) 
print(output)
print(input)
P = F.softmax(input,1)
print(P)
Q = -torch.log(P)
#Q = -P
print(Q)
(Q[0,target[0]]+Q[1,target[1]]+Q[2,target[2]])/3

### Question 13: why do we compute the loss only for a subset of labels?

#### Answer:

We compute the loss only for the labels we know in advance. The other labels are unknown (semi-supervised learning). The filtering is important to establish a relationship between all the nodes.

### Question 14: when we are training the neural network, the inputs "inputs" are constant or not?

#### Answer:

They are not constant. They depend on the embedding layer which is also learned during the training.

In [ ]:
# Train the neural network and the embedding layer
all_logits = []
for epoch in range(50):
    logits = net(G, inputs)
    # we save the logits for visualization later
    all_logits.append(logits.detach())
    logp = F.log_softmax(logits, 1) # the "1" indicates the dimension along which log_softmax will be computed.
    # we compute loss for labeled nodes only
    loss = F.nll_loss(logp[labeled_nodes], labels) # "labels" is a vector of integers (each integer corresponds to a class indentifier between 0 and K-1)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print('Epoch %d | Loss: %.4f' % (epoch, loss.item()))

This is a rather toy example, so it does not even have a validation or test
set. Instead, since the model produces an output feature of size 2 for each node, we can
visualize by plotting the output feature in a 2D space.
The following code animates the training process from initial guess
(where the nodes are not classified correctly at all) to the end
(where the nodes are linearly separable).



In [ ]:
import matplotlib.animation as animation
import matplotlib.pyplot as plt

def draw(i):
    cls1color = '#00FFFF'
    cls2color = '#FF00FF'
    pos = {}
    colors = []
    for v in range(34):
        pos[v] = all_logits[i][v].numpy()
        cls = pos[v].argmax()
        colors.append(cls1color if cls else cls2color)
    ax.cla()
    ax.axis('off')
    ax.set_title('Epoch: %d' % i)
    nx.draw_networkx(nx_G.to_undirected(), pos, node_color=colors,
            with_labels=True, node_size=300, ax=ax)

fig = plt.figure(dpi=150)
fig.clf()
ax = fig.subplots()
draw(0)  # draw the prediction of the first epoch
plt.close()

In [ ]:
ani = animation.FuncAnimation(fig, draw, frames=len(all_logits), interval=200)
from IPython.display import HTML
# If 'ffmpeg' is not installed on your laptop, the following command fails
# Just run this notebook on Google Colab to see the animation
HTML(ani.to_html5_video())



## Step 6: Create your own GCN layer
--------------------------------------------------

In the following cells, you will code your own GCN layer based on the famous message passing paradigm.
This approach is especially relevant for large graphs.



### Question 15: what is the main advantage of message-passing paradigm?

#### Answer:

Message-passing involvesd a message function defined on each edge to generate a “message” 
by combining the edge feature with node features at its two ends.

Then, an update function is defined on each node to update the node feature by aggregating 
its incoming messages using a reduce operation (sum, average, max, etc.). 

This way to ccommunicate between nodes by using the edges is well adapted to graphs, especially when the graph contains many nodes and edges.

#### Define the message and reduce function

In DGL, the message functions are expressed as Edge UDFs (User Defined Function). 
Edge UDFs take in a single argument edges. It has three members src, dst, and data for accessing source node features, destination node features, and edge features. 
Here, the function computes messages only from source node features.

Hence, by default, the messages come from the neighborhood of the nodes, i.e. they arrive via the 1-hop nodes connected via an edge.

We first define the message and reduce functions. Since the aggregation on a node only involves summing over the neighbors’ representations, we can simply use builtin functions.

### Question 16: what is the goal of "fn.copy_u" and "fn.sum" in the next cell?

#### Answer:

dgl.function.copy_u(u, out) is a builtin message function that computes message using source node feature "u" is the source node feature (wrt to the edge) and "out" is the message name.
The message are stored in a "mailbox" associated to the node.

See more details on https://docs.dgl.ai/en/latest/guide/message-api.html?highlight=mailbox

dgl.function.sum is a builtin reduce function that aggregates all messages by sum. 
The result is stored in the node feature "h"
Generally the messages come from the neighborhood of the node.


In [ ]:
import dgl.function as fn

gcn_message = fn.copy_u(u='h', out='m') 

gcn_reduce = fn.sum(msg='m', out='h')

We then proceed to define the GCNLayer module. A GCNLayer essentially performs message passing on all the nodes then applies a fully-connected layer.

In [ ]:
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, feature):
        # Creating a local scope so that all the stored ndata and edata
        # (such as the `'h'` ndata below) are automatically popped out
        # when the scope exits.
        #
        # Note that an input is considered as a node feature.
        with g.local_scope():
            g.ndata['h'] = feature
            # "update_all" collect features from source nodes and aggregate them in destination nodes
            g.update_all(gcn_message, gcn_reduce)
            h = g.ndata['h']
            return self.linear(h)

### Question 17: create a class GCNmp similar to the previous class GCN which is based on the GCNlayer classes. 

In [ ]:
# Fill in the cell

# Define a 2-layer GCN model based on GCNlayer
# GCNmp with mp = message-passing
class GCNmp(nn.Module):
    def __init__(self, in_feats, hidden_size, num_classes):
        super(GCNmp, self).__init__()
        self.gcn1 = GCNLayer(in_feats, hidden_size)
        self.gcn2 = GCNLayer(hidden_size, num_classes)

    def forward(self, g, inputs):
        h = self.gcn1(g, inputs)
        h = torch.relu(h)
        h = self.gcn2(g, h)
        return h


Initialization

In [ ]:
netmp = GCNmp(embed_dim, hidden_dim, 2)

embedmp = nn.Embedding(Nnodes, embed_dim)  
inputs_mp = embedmp.weight

optimizer = torch.optim.Adam(itertools.chain(netmp.parameters(), embedmp.parameters()), lr=0.01)

In [ ]:
all_logits = []
for epoch in range(30):
    logits = netmp(G, inputs_mp)
    # we save the logits for visualization later
    all_logits.append(logits.detach())
    logp = F.log_softmax(logits, 1)
    # we only compute loss for labeled nodes
    loss = F.nll_loss(logp[labeled_nodes], labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print('Epoch %d | Loss: %.4f' % (epoch, loss.item()))